# Supporting functions

In [1]:
import pandas as pd
import operator

from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

/home/sunil.patel/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# function for common string cleanup
def cleanup(x):
    return str(x).replace("\t","").replace("\s","").strip()

# Reading mater drug file and creating dictionary
returning relevent result to the entered one

In [3]:
df = pd.read_csv("data/out_unique_final.csv", sep="ж")
df = df.apply(lambda x: x.astype(str).str.lower())

/home/sunil.patel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
df.columns

Index(['S/N', 'Manufacturer', 'Name', 'Type', 'Unit(Dose)',
       'Constituents/Unit', 'Package unit', 'Price(In Rs.)',
       'Price/Unit (In Rs.)'],
      dtype='object')

In [5]:
class master_drug_file_handeller:
    max_replacements = 10
    def __init__(self):
        self.master_drug_list = []
        self.master_drug_store = []
        for i in range(0, len(df)):
            drug_dict = df.iloc[i].to_dict()
            drug_dict['id'] = i
            self.master_drug_list.append(drug_dict['Constituents/Unit'])
            self.master_drug_store.append(drug_dict)
    
    def relevent_searcher(self, text):
        self.relevence_dict = {}
        scores = [fuzz.ratio(i, text) for i in self.master_drug_list]
        scores_ids = [i for i in range(0,len(self.master_drug_list))]
        for score,id in zip(scores, scores_ids):
            if id not in self.relevence_dict.keys():
                self.relevence_dict[id] = score
        #soriting the suggestion    
        sorted_x = sorted(self.relevence_dict.items(), key=operator.itemgetter(1), reverse=True)
        return  sorted_x[:10]

In [6]:
MDFH = master_drug_file_handeller()

In [25]:
relevence_dict_list = MDFH.relevent_searcher("PIPERACILLIN+TAZOBACTUM".lower())

In [29]:
MDFH.master_drug_store[37032]

{'Constituents/Unit': 'piperacillin+tazobactam',
 'Manufacturer': 'profic organic ltd.',
 'Name': 'proact (4000+500)',
 'Package unit': '1 vial',
 'Price(In Rs.)': '189.00',
 'Price/Unit (In Rs.)': '189.00',
 'S/N': '7',
 'Type': 'injection',
 'Unit(Dose)': '1 vial',
 'id': 37032}

# Reading file provided by Janki - Patient Data

---

### Supporting function

In [10]:
import random
import time
import string

In [11]:
def generate_random_age(min, max):
    random_age = random.randint(min, max)
    return int(random_age)

In [12]:
def generate_random_patient_Name(stringLength=2):
    """
    generate_random_patient_Name
    """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength)).upper()

In [13]:
def generate_sex():
    """
    generate_sex()
    """
    random_prob = random.random()
    if random_prob > 0.66:
        return 2
    else: 
        return 1

In [14]:
def generate_weight():
    """
    generate_weight()
    """
    random_prob = random.random()
    if random_prob < 0.1:
        #generating probability for age under 18
        return random.randint(15,21)
    if random_prob >= 0.1  and  random_prob <= 0.9 :
        # retuen None as many of the values are none
        return None
    elif random_prob >= 0.9 :
         return random.randint(21,78)


In [15]:
def generate_weight():
    "always not recorded"
    return None
def generate_BMI():
    "always not recorded"
    return None
def generate_antibiotics_in_past():
    """
    Assuming 20% time patient dont know or taking patients taking antibiotics for the first time
    """
    random_prob = random.random()
    if random_prob < 0.1:
        # 0.1 % person knws tahat they have takenn antibiotics in the past
        return "Yes"
    if random_prob >= 0.1  and  random_prob <= 0.9 :
        # not sure patients
        return None
    elif random_prob >= 0.9 :
        # 0.1 % person knws tahat they have  not takenn antibiotics in the past
        return "No"
def generate_details_of_antibiotics_used():
    """
    Always render no as no one rememberes waht was taken last time
    """
    return "Dont Know"

def generate_num_day_hospitalized(min, max):
    """
    randomly generatiing number of days stayed in hospital  
    """
    return random.randint(min, max)
def generate_culture_sensitivity_test():
    random_prob = random.random()
    if random_prob>0.90:
        return "Yes"
    else:
        return "No"


### Antibiotic group genration

In [39]:
# getting/antibiotic_class
antibiotic_mapping = pd.read_excel('data/antibiotic_class.xlsx',sheetname='Sheet1')

In [55]:
antibitic_class_mapping = {}
for i in range(0,len(antibiotic_mapping)):
    dict_ = antibiotic_mapping.iloc[i].to_dict()
    antibitic_class_mapping[dict_['Drug Name'].lower().replace("/",'+').strip()] = dict_['Drug class'].lower().strip()

In [69]:
def generate_details_for_antibiotics(antibiotic_generic_name):
    relevence_dict_list = MDFH.relevent_searcher(antibiotic_generic_name.lower())
    print(relevence_dict_list)
    random_chosen  = random.randint(0, len(relevence_dict_list))
    random_antbiotic_id  = relevence_dict_list[random_chosen][0]
    return MDFH.master_drug_store[random_antbiotic_id]

def get_antibiotic_class(antibiotic):
    if '+' in antibiotic:
        return "Fixed dose combiination"
    try:
        class_  = antibitic_class_mapping[str(antibiotic).lower()]
        return class_
    except:
        return "**NA**"
    
def generate_antibiotic_group(generic_name,group_no=1):
    new_dict = {}
    """
     '(Antibiotic 1) Generic Name ',
     '(Antibiotic 1) Class',
     '(Antibiotic 1) Dose (mg)',
     '(Antibiotic 1) Route',
     '(Antibiotic 1) Units',
     '(Antibiotic 1) Total Dose Administered (mg)',
     '(Antibiotic 1) Cost (Rs)',
     '(Antibiotic 1) Total Cost (Rs)',
    """
    dict_ = dict(generate_details_for_antibiotics(generic_name))
    dict_["Class"] = get_antibiotic_class(generic_name)
    for key, value in dict_.items():
        new_dict[key+str("_")+str(group_no)] =  value

    
    return new_dict

In [70]:
generate_antibiotic_group("cefixime")

[(13216, 100), (13217, 100), (13218, 100), (13219, 100), (13220, 100), (13221, 100), (13222, 100), (13223, 100), (13225, 100), (13226, 100)]


{'Class_1': 'cephalosporin',
 'Constituents/Unit_1': 'cefixime',
 'Manufacturer_1': 'hygeia pharmaceuticals',
 'Name_1': 'valcef (100 mg)',
 'Package unit_1': '10 tablet',
 'Price(In Rs.)_1': '11.00',
 'Price/Unit (In Rs.)_1': '1.10',
 'S/N_1': '3',
 'Type_1': 'tablet',
 'Unit(Dose)_1': '1 tablet',
 'id_1': 13218}

---

In [18]:
patient_data = pd.read_excel('data/xcel_patient_data.xlsx',sheetname='Sheet1&3')

In [19]:
list(patient_data.columns)

['Form No.',
 'Hosp. Name',
 'Pt. Name/ IP no',
 'Sex',
 'Age (in yr)',
 'Weight (in kg)',
 'BMI (kg/m2)',
 'Antibiotic in Past',
 'If Yes, Details of Antibiotic used',
 'No. of Days Hospitalized',
 'Diagnosis',
 'Culture SensitIVity Test',
 'No. of Drugs/prescription',
 'No. of Antibiotics',
 'No. of Antibiotics by Generic Name',
 'No. of Antibiotics by Brand Name',
 'No. of FDC',
 'No. of Antibiotics prescribed from NLEM 2017',
 'Switch Over therapy',
 'Parentral',
 'Oral',
 'Syrup',
 'Eye drop',
 'Ear drop',
 'Others formulations',
 '(Antibiotic 1) Generic Name ',
 '(Antibiotic 1) Class',
 '(Antibiotic 1) Dose (mg)',
 '(Antibiotic 1) Route',
 '(Antibiotic 1) Units',
 '(Antibiotic 1) Total Dose Administered (mg)',
 '(Antibiotic 1) Cost (Rs)',
 '(Antibiotic 1) Total Cost (Rs)',
 '(Antibiotic 2) Generic Name',
 '(Antibiotic 2) Class',
 '(Antibiotic 2) Dose (mg)',
 '(Antibiotic 2) Route',
 '(Antibiotic 2) Units',
 '(Antibiotic 2) Total Dose Administered (mg)',
 '(Antibiotic 2) Cost (Rs)

In [20]:
patient_data['If Yes, Details of Antibiotic used']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     AKT
27     NaN
28     NaN
29     NaN
      ... 
102    NaN
103    NaN
104    NaN
105    NaN
106    NaN
107    NaN
108    NaN
109    NaN
110    NaN
111    NaN
112    NaN
113    NaN
114    NaN
115    NaN
116    NaN
117    NaN
118    NaN
119    NaN
120    NaN
121    NaN
122    NaN
123    NaN
124    NaN
125    NaN
126    NaN
127    NaN
128    NaN
129    NaN
130    NaN
131    NaN
Name: If Yes, Details of Antibiotic used, Length: 132, dtype: object